In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2

import glob
import matplotlib.pyplot as plt

import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dense, BatchNormalization, ReLU, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Add, GlobalAveragePooling2D, concatenate#, Average, Concatenate, SpatialDropout2D
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras import Input,models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

import imgaug as ia
import imgaug.augmenters as iaa

from google.colab import drive
from google.colab import files


In [ ]:
drive.mount("/content/drive", force_remount=True)
%cd drive/MyDrive/Colab Notebooks/

In [ ]:
label_encoder = {"Basketball" : 0, "Football" : 1, "Rowing" : 2, "Swimming" : 3, "Tennis" : 4, "Yoga" : 5}

In [ ]:
augmentation = iaa.Sequential([
    # 1. Flip
    iaa.Fliplr(0.5),
    # 2. Affine
    iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
               rotate=(-10, 10),
               scale=(0.8, 1.2)),
               
    # 3. Multiply
    iaa.Multiply((0.8, 1.2)),
    # 4. Linearcontrast
    iaa.LinearContrast((0.8, 1.2)),
    # Perform methods below only sometimes
    iaa.Sometimes(0.7,
        # 5. GaussianBlur
        iaa.GaussianBlur((0.1, 0.6))
        )
])


In [ ]:
imgsP1 = glob.glob(f"./NN_DataSet/Train/*")
TrainImgs = []
TrainLabel = []
# x = 325 , y = 420
for f in imgsP1: 
  img = cv2.imread(f)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # img = img.astype(np.float32)
  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)

  l = label_encoder[f.split('/')[3].split('_')[0]]

  TrainImgs.append(img)#/255
  TrainLabel.append(l)
  # n = 3 if l == 0 else 2
  # for i in range(n):
  #   augmented_img = augmentation(image=img)
  #   TrainImgs.append(augmented_img/255)
  #   TrainLabel.append(l)


TrainImgs = np.array(TrainImgs)
TrainLabel = np.reshape(TrainLabel,(-1,1))
del imgsP1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(TrainImgs, TrainLabel, test_size=0.2, random_state=10)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    rescale=1./255,
    width_shift_range=[-0.05, 0.05],
    height_shift_range=[-0.05, 0.05],
    horizontal_flip=True,
    brightness_range=[0.9,1.1],
    zoom_range=[-0.1,0.1],
    # fill_mode = "wrap"
    )

In [ ]:
datagen.fit(X_train)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
del TrainImgs
del TrainLabel

In [ ]:
plt.imshow(X_train[77,:])

In [ ]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

In [ ]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (256, 256, 3))
  X = BatchNormalization()(input_layer)


  X = Conv2D(filters = 48, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')( X)

  X = MaxPooling2D(pool_size = 2, strides = 2)(X)

  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  X = Conv2D(filters = 96, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  X = MaxPooling2D(pool_size= 2, strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 32, f2_conv1 = 48, f2_conv3 = 64, f3_conv1 = 16, f3_conv5 = 16, f4 = 16)

  # 2nd Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 64, f2_conv3 = 96, f3_conv1 = 16, f3_conv5 = 48, f4 = 32)

  X = MaxPooling2D(pool_size= 2, strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 96, f2_conv1 = 48, f2_conv3 = 104, f3_conv1 = 16, f3_conv5 = 24, f4 = 32)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 64, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(128, activation = 'relu')(X1)
  X1 = Dropout(0.2)(X1)
  X1 = Dense(6, activation = 'softmax', name = 'x1')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 80, f2_conv1 = 56, f2_conv3 = 112, f3_conv1 = 24, f3_conv5 = 32, f4 = 32)

  # 5th Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 64, f2_conv3 = 128, f3_conv1 = 24, f3_conv5 = 32, f4 = 32)

  # 6th Inception block
  X = Inception_block(X, f1 = 56, f2_conv1 = 112, f2_conv3 = 144, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 64, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(128, activation = 'relu')(X2)
  X2 = Dropout(0.2)(X2)
  X2 = Dense(6, activation = 'softmax', name = 'x2')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 80, f2_conv3 = 160, f3_conv1 = 16, 
                      f3_conv5 = 64, f4 = 64)

  X = MaxPooling2D(pool_size = 2, strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 80, f2_conv3 = 160, f3_conv1 = 16, f3_conv5 = 64, f4 = 64)

  # 9th Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 192, f3_conv1 = 48, f3_conv5 = 64, f4 = 64)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.1)(X)

  # output layer 
  X = Dense(6, activation = 'softmax', name = 'GoogLeNetOutPut')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
GoogLeNetModel = GoogLeNet()
GoogLeNetModel.summary()

In [ ]:
GoogLeNetModel.compile(optimizer=Adam(learning_rate=0.001,)
,loss=SparseCategoricalCrossentropy() ,metrics=["accuracy"])#,metrics=["accuracy"]

In [ ]:
GoogLeNetModel.fit(datagen.flow(X_train, y_train, batch_size=16,),
         steps_per_epoch=len(X_train) / 16, epochs=150)

In [ ]:
GoogLeNetModel.fit(X_train, y_train, batch_size=64, epochs=32)

In [ ]:
GoogLeNetModel.save('models/v_6')

In [ ]:
GoogLeNetModel = models.load_model('models/v_6')

In [ ]:
Ptrain = GoogLeNetModel.predict(X_train)
Ptrain= Ptrain[0]
s1 = np.sum(np.argmax(Ptrain, axis=1) == y_train[:,0])/Ptrain.shape[0]

# del Ptrain
# del X_train
# del y_train
s1

In [ ]:
Ptest = GoogLeNetModel.predict(X_test)
Ptest= Ptest[0]
s2 = np.sum(np.argmax(Ptest, axis=1) == y_test[:,0])/Ptest.shape[0]

# del Ptest
# del X_test
# del y_test
s2

In [ ]:
imgsP2 = glob.glob(f"./NN_DataSet/Test/*")
TestImgs = []
testID = []
# x = 500 , y = 700
for f in imgsP2:
  img = cv2.imread(f)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img.astype(np.float32)
  img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)


  img /= 255
  TestImgs.append(img)
  testID.append(f.split('/')[3])

TestImgs = np.array(TestImgs)
del imgsP2

In [ ]:
tests = GoogLeNetModel.predict(TestImgs)
tests=tests[0]
tests = np.argmax(tests, axis=1)
col1 = pd.DataFrame(testID,columns=["image_name"]).reset_index(drop=True)
col2 = pd.DataFrame(tests,columns=["label"]).reset_index(drop=True)
sub = pd.concat([col1,col2],axis=1)#.reset_index(drop=True)
sub.to_csv("./submission6.csv",index=False)

In [ ]:
i=55

plt.title(str(tests[i]))
plt.imshow(TestImgs[i,:])

In [ ]:
i=230

plt.title(str(tests[i]))
plt.imshow(TestImgs[i,:])